In [1]:
pip install linearmodels

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from linearmodels import IV2SLS, IVLIML, IVGMM

In [3]:
file="https://github.com/VitorKamada/ECO7110/raw/master/Data/mus06data.dta"
data = pd.read_stata(file)
data.head()

,ssiratio,age,educyr,drugexp,private,female,hisp,marry,vegood,good,fair,poor,poverty,lowincome,midincome,msa,firmsz,multlc,income,priolist,totchr,black,hi_empunion,ldrugexp,age2,linc,vgh,fph,blhisp,_est_reg1,_est_reg2,_est_reg3,_est_reg4,_est_OLS,_est_ivreg2_0,_est_ivreg2_1,_est_ivreg2_2,_est_ivreg2_3
0,0.149877,74,16,595,1,1,0,1,0,1,0,0,0,0,0,0,0.0,0,94.545891,1,0,0,1,6.388561,5476.0,4.549085,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1
1,0.395856,73,8,1783,1,0,0,1,0,1,0,0,0,0,0,0,0.1,0,35.796379,1,3,0,1,7.486053,5329.0,3.577847,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1
2,1.000000,80,12,176,0,1,0,0,0,0,0,0,1,0,0,1,0.0,0,9.600000,1,1,0,0,5.170484,6400.0,2.261763,0.0,0.0,0.0,1,1,1,1,1,1,1,1,1
3,0.206639,70,17,2437,1,0,0,1,0,1,0,0,0,0,0,1,0.0,0,38.714771,1,5,0,1,7.798523,4900.0,3.656221,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1
4,0.537192,91,16,330,0,0,0,1,1,0,0,0,0,0,1,1,0.0,0,8.725000,1,3,0,0,5.799093,8281.0,2.166193,1.0,0.0,0.0,1,1,1,1,1,1,1,1,1


In [4]:
controls = ['totchr', 'female', 'age', 'linc','blhisp']
print(data[['ldrugexp','hi_empunion'] + 
           controls].describe(percentiles=[]))

           ldrugexp   hi_empunion  ...          linc        blhisp
count  10391.000000  10391.000000  ...  10089.000000  10391.000000
mean       6.479664      0.379655  ...      2.743271      0.170340
std        1.363393      0.485324  ...      0.913144      0.375956
min        0.000000      0.000000  ...     -6.907755      0.000000
50%        6.677083      0.000000  ...      2.743160      0.000000
max       10.180172      1.000000  ...      5.744476      1.000000

[6 rows x 7 columns]


In [5]:
instruments = ['ssiratio', 'lowincome', 'multlc', 'firmsz']
print(data[instruments].describe(percentiles=[]))

           ssiratio     lowincome        multlc        firmsz
count  10391.000000  10391.000000  10391.000000  10391.000000
mean       0.520628      0.185641      0.060341      0.136501
std        0.374589      0.388836      0.238128      2.138914
min       -2.100647      0.000000      0.000000      0.000000
50%        0.490139      0.000000      0.000000      0.000000
max        9.250620      1.000000      1.000000     50.000000


In [6]:
data[['hi_empunion'] + instruments].corr()

,hi_empunion,ssiratio,lowincome,multlc,firmsz
hi_empunion,1.000000,-0.196284,-0.114425,0.119054,0.037173
ssiratio,-0.196284,1.000000,0.249820,-0.174394,-0.040481
lowincome,-0.114425,0.249820,1.000000,-0.060701,-0.007802
multlc,0.119054,-0.174394,-0.060701,1.000000,0.187394
firmsz,0.037173,-0.040481,-0.007802,0.187394,1.000000


In [0]:
from statsmodels.api import OLS, add_constant
data['const'] = 1
controls = ['const'] + controls

In [8]:
ivolsmod = IV2SLS(data.ldrugexp, 
     data[['hi_empunion'] + controls], None, None)
res_ols = ivolsmod.fit()
print(res_ols)

                            OLS Estimation Summary                            
Dep. Variable:               ldrugexp   R-squared:                      0.1770
Estimator:                        OLS   Adj. R-squared:                 0.1765
No. Observations:               10089   F-statistic:                    2262.6
Date:                Wed, Jul 31 2019   P-value (F-stat)                0.0000
Time:                        19:17:46   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
hi_empunion     0.0739     0.0260     2.8441     0.0045      0.0230      0.1248
const           5.8611     0.1570     37.320    

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [9]:
ivmod = IV2SLS(data.ldrugexp, data[controls],
  data.hi_empunion, data[['ssiratio','multlc']])
res_2sls = ivmod.fit()
print(res_2sls.summary)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          IV-2SLS Estimation Summary                          
Dep. Variable:               ldrugexp   R-squared:                      0.0414
Estimator:                    IV-2SLS   Adj. R-squared:                 0.0409
No. Observations:               10089   F-statistic:                    1955.4
Date:                Wed, Jul 31 2019   P-value (F-stat)                0.0000
Time:                        19:17:46   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           6.8752     0.2579     26.660     0.0000      6.3697      7.3806
totchr          0.4512     0.0103     43.769    

In [10]:
ivmod = IVGMM(data.ldrugexp, data[controls], 
  data.hi_empunion, data[['ssiratio','multlc']])
res_gmm = ivmod.fit()
print(res_gmm)

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


                          IV-GMM Estimation Summary                           
Dep. Variable:               ldrugexp   R-squared:                      0.0406
Estimator:                     IV-GMM   Adj. R-squared:                 0.0400
No. Observations:               10089   F-statistic:                    1952.6
Date:                Wed, Jul 31 2019   P-value (F-stat)                0.0000
Time:                        19:17:47   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           6.8778     0.2580     26.658     0.0000      6.3722      7.3835
totchr          0.4510     0.0103     43.738    

In [11]:
print(res_gmm.first_stage)

    First Stage Estimation Results    
                           hi_empunion
--------------------------------------
R-squared                       0.0794
Partial R-squared               0.0215
Shea's R-squared                0.0215
Partial F-statistic             117.32
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(2)
========================== ===========
const                           0.9834
                              (16.780)
totchr                          0.0133
                              (3.6234)
female                         -0.0727
                             (-7.5644)
age                            -0.0081
                             (-11.311)
linc                            0.0444
                              (6.7838)
blhisp                         -0.0679
                             (-5.5484)
ssiratio                       -0.1823
                             (-7.8326)
multlc                          0.1209
                         

In [12]:
ivmod = IVLIML(data.ldrugexp, data[controls], 
               data.hi_empunion, data[['ssiratio','multlc']])
res_liml = ivmod.fit(cov_type='robust')
from linearmodels.iv.results import compare
print(compare({'2SLS':res_2sls,'LIML': res_liml, 'GMM': res_gmm}))

                       Model Comparison                       
                              2SLS           GMM          LIML
--------------------------------------------------------------
Dep. Variable             ldrugexp      ldrugexp      ldrugexp
Estimator                  IV-2SLS        IV-GMM       IV-LIML
No. Observations             10089         10089         10089
Cov. Est.                   robust        robust        robust
R-squared                   0.0414        0.0406        0.0400
Adj. R-squared              0.0409        0.0400        0.0394
F-statistic                 1955.4        1952.6        1952.3
P-value (F-stat)            0.0000        0.0000        0.0000
==================     ===========   ===========   ===========
const                       6.8752        6.8778        6.8807
                          (26.660)      (26.658)      (26.577)
totchr                      0.4512        0.4510        0.4513
                          (43.769)      (43.738)      (

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [13]:
res_gmm.j_stat

H0: Expected moment conditions are equal to 0
Statistic: 1.0475
P-value: 0.3061
Distributed: chi2(1)
WaldTestStatistic, id: 0x7f99d2b90128

In [14]:
ivmod = IVGMM(data.ldrugexp, data[controls],
        data.hi_empunion, data[instruments])
res_gmm_all = ivmod.fit()

/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:478: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [15]:
print(res_gmm_all)

                          IV-GMM Estimation Summary                           
Dep. Variable:               ldrugexp   R-squared:                      0.0829
Estimator:                     IV-GMM   Adj. R-squared:                 0.0824
No. Observations:               10089   F-statistic:                    2042.1
Date:                Wed, Jul 31 2019   P-value (F-stat)                0.0000
Time:                        19:17:49   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                              Parameter Estimates                              
             Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------
const           6.7126     0.2426     27.670     0.0000      6.2371      7.1881
totchr          0.4495     0.0100     44.738    

In [16]:
print(res_gmm_all.first_stage)

    First Stage Estimation Results    
                           hi_empunion
--------------------------------------
R-squared                       0.0821
Partial R-squared               0.0243
Shea's R-squared                0.0243
Partial F-statistic             179.47
P-value (Partial F-stat)        0.0000
Partial F-stat Distn           chi2(4)
========================== ===========
const                           0.9899
                              (16.959)
totchr                          0.0133
                              (3.6494)
female                         -0.0724
                             (-7.5497)
age                            -0.0080
                             (-11.206)
linc                            0.0410
                              (6.3552)
blhisp                         -0.0676
                             (-5.5369)
ssiratio                       -0.1690
                             (-7.3289)
lowincome                      -0.0637
                         

In [17]:
res_gmm_all.j_stat

H0: Expected moment conditions are equal to 0
Statistic: 11.5903
P-value: 0.0089
Distributed: chi2(3)
WaldTestStatistic, id: 0x7f99d2bfccc0